# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 6 Assignment: Classification Neural Network**

**Student Name: Rebekah Griesenauer**

# Assignment Instructions

For this assignment you will use the **cls-32-spring-2018.csv** dataset.  This is a dataset that I generated specifically for this semester.  You can find the CSV file in the **data** directory of the class GitHub repository here: [cls-32-spring-2018.csv](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/data/cls-32-spring-2018.csv).

You will train 5 neural networks, one for each fold of a 5-fold cross validation and return the out of sample predictions.  You will submit these perdictions to the **submit** function.  See [Assignment #1](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class1.ipynb) for details on how to submit an assignment or check that one was submitted.

Complete the following tasks:

* Normalize all numerics to zscores and all text/categoricals to dummies.  Normalize the *target* to an index (use *encode_text_index*).
* Your target (y) is the filed named *target*.
* If you find any missing values (NA's), replace them with the median values for that column.
* This is a larger dataset with rows of nearly 100K.  Training will take longer but it should not take you longer than 20 minutes or so to train.  If it is taking longer, use a more simple neural network.
* To save some time, modify the training set to be short, say 100 lines, and use that to debug your code and use the full set the end.  No need to waste 20 minutes just to see you have an error generating the submit data frame.
* Use a 5-fold cross validation and return out of sample predictions.  
* Your submission should contain the id (column name *id*), your prediction (column name *pred*), the expected value (from the **cls-32-spring-2018.csv** dataset. Note that both *pred* and *y* should be in index form (i.e. 1,2,3,4 etc, not the text form, TA-0, TA-1, etc)
* You might get warnings about the means of your columns differing from mine.  Do not worry about small differences.  
* Your submitted dataframe will have these columns: id, y, pred.


# Helpful Functions

You will see these at the top of every module and assignment.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [14]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))
        
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Assignment #6 Sample Code

The following code provides a starting point for this assignment.

In [23]:
import os
import pandas as pd
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

key = "r7Th7Lhul2aj0C93lQNZkA9QZFuc6RW1YfPO4QM2"  # This is an example key and will not work.
file = '/Users/rebekahgriesenauer/Documents/AI/deep_neural_networks/assignments/assignment_griesenauer_class6.ipynb'

# Begin assignment
path = "../data/"

filename_read = os.path.join(path,"cls-32-spring-2018.csv")
df = pd.read_csv(filename_read)

ids = df['id']
# Shuffle
np.random.seed(42)
df = df.reindex(np.random.permutation(df.index))
df.reset_index(inplace=True, drop=True)
df.drop('id',1,inplace=True)

In [25]:
#feature engineering
# Encode the feature vector

#Check to see what columns have null values
missing_value_columns = df.columns[df.isna().any()].tolist()
#replace missing values with median
missing_median(df, 'weight')

#find columns with numeric data
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
#encode numeric columns as z-scores
for column in numeric_columns:
    encode_numeric_zscore(df, column, mean=None, sd=None)
    
#find columns with non-numeric data
text_columns = df.select_dtypes(include = object).columns.tolist()
#remove target 
text_columns.remove('target')

#ecode text/category items to dummies
for text_col in text_columns:
    encode_text_dummy(df, text_col)
    

#encode target column as index
target = encode_text_index(df,'target')

In [40]:
#classification neural network

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
x,y = to_xy(df, 'target')

# Cross-Validate
kf = KFold(5)
    
oos_y = []
oos_pred = []
fold = 0

for train, test in kf.split(x):
    fold+=1
    print("Fold #{}".format(fold))
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(100, input_dim=x.shape[1], activation='relu')) # Hidden 1
    model.add(Dense(50, activation='relu')) # Hidden 2
    model.add(Dense(25, activation='relu')) # Hidden 3
    model.add(Dense(y.shape[1],activation='softmax')) # Output
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')

    model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor],verbose=0,epochs=1000)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)
    oos_pred.append(pred)        

    # Measure this fold's accuracy
    y_compare = np.argmax(y_test,axis=1) # For accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print("Fold score (accuracy): {}".format(score))


# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)

oos_pred = np.concatenate(oos_pred)
oos_y_compare = np.argmax(oos_y,axis=1) # For accuracy calculation

score = metrics.accuracy_score(oos_y_compare, oos_pred)
print("Final score (accuracy): {}".format(score))    
    
# Write the cross-validated prediction
#oos_y = pd.DataFrame(oos_y)
#oos_pred = pd.DataFrame(oos_pred)
#oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:73: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Fold #1
Epoch 00137: early stopping
Fold score (accuracy): 0.9926663174436878
Fold #2
Epoch 00091: early stopping
Fold score (accuracy): 0.9879518072289156
Fold #3
Epoch 00114: early stopping
Fold score (accuracy): 0.9963321980613047
Fold #4
Epoch 00120: early stopping
Fold score (accuracy): 0.9963321980613047
Fold #5
Epoch 00106: early stopping
Fold score (accuracy): 0.9965941839140686
Final score (accuracy): 0.9939749567768639


array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [50]:
y_compare = np.argmax(oos_y, axis=1)
submit_df = pd.DataFrame();
submit_df['id']=ids
submit_df['y'] = y_compare
submit_df['pred']= oos_pred
submit_df

,id,y,pred
0,1,2,2
1,2,0,0
2,3,1,1
3,4,1,1
4,5,1,1
5,6,2,2
6,7,2,2
7,8,4,4
8,9,4,4
9,10,4,4


In [51]:
# Save a copy 
submit_df.to_csv('6.csv',index=False)

# Submit the assignment
submit(source_file=file,data=submit_df,key=key,no=6)

Success: Submitted Assignment #6 for rebekah.griesenauer:
This is your first submission of this assignment.

